In [1]:
import pdfplumber
def extract_resume_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

In [2]:
resume_text = extract_resume_text("C:/Users/kambh/Downloads/resume1.pdf.pdf")

In [4]:
resume_text

'Kambhampati Sruthi\nCONTACT Phone: +91 8978791505\nEmail:kambhampatisruthi@gmail.com\nLinkedin: www.linkedin.com/in/kambhampati-sruthi\nOBJECTIVE Aspiring AI and Data Science engineer seeking internship opportunities to apply skills in\nmachine learning, data analysis, and intelligent systems development.\nEDUCATION B. Tech -Artificial Intelligence and Data Science ,Ramachandra College Of Engineering\n2023-2027\nCGPA:9.4\nIntermediate\nNRI Junior College, Eluru\n2021–2023 | Marks: 95%\nSSC (Class X)\nIndian High EM School, Mulagalampadu\n2020–2021 | GPA: 9.8\nOBJECTIVE Programming languages : Python, c\nDatabase : SQL\nTools : VS Code, Github\nC programming getting started 1 - UC Santa Cruz\nCERTIFICATES Introduction to c programming- UC Santa Cruz\nPython for beginners - Simplilearn\nDate of Birth:11/06/2005\nPERSONAL\nNationality: Indian\nDETAILS\nLanguages known: English,Telugu'

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
def extract_keywords(job_description):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform([job_description])
    keywords = vectorizer.get_feature_names_out()
    return list(set(keywords))

In [8]:
job_description = """
We are looking for a Python developer with experience in SQL.
Knowledge of Git is a plus.
"""
job_keywords = extract_keywords(job_description)
print("📌 Extracted Keywords:", job_keywords)

📌 Extracted Keywords: ['experience', 'looking', 'git', 'knowledge', 'python', 'sql', 'plus', 'developer']


In [9]:
def find_missing_keywords(resume_text, job_keywords):
    resume_words = set(resume_text.lower().split())
    missing = [kw for kw in job_keywords if kw.lower() not in resume_words]
    return missing

In [13]:
resume_text = extract_resume_text("C:/Users/kambh/Downloads/resume1.pdf.pdf")
job_keywords = extract_keywords(job_description)
missing_keywords = find_missing_keywords(resume_text, job_keywords)
print("❌ Missing Keywords:", missing_keywords)

❌ Missing Keywords: ['experience', 'looking', 'git', 'knowledge', 'plus', 'developer']


In [15]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
import io
def append_keywords_to_pdf(original_pdf_path, output_pdf_path, missing_keywords):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont("Helvetica-Bold", 14)
    can.drawString(100, 800, "Optimised Skills")
    can.setFont("Helvetica", 12)
    y = 780
    for kw in missing_keywords:
        can.drawString(120, y, f"• {kw}")
        y -= 20
        if y < 50:
            can.showPage()
            y = 800
    can.save()
    packet.seek(0)
    original_pdf = PdfReader(original_pdf_path)
    new_pdf = PdfReader(packet)
    writer = PdfWriter()
    for page in original_pdf.pages:
        writer.add_page(page)
    for page in new_pdf.pages:
        writer.add_page(page)
    with open(output_pdf_path, "wb") as f:
        writer.write(f)
    print(f"✅ Optimised resume saved as: {output_pdf_path}")

In [17]:
append_keywords_to_pdf(
    original_pdf_path="C:/Users/kambh/Downloads/resume1.pdf.pdf",
    output_pdf_path="C:/Users/kambh/Downloads/optimised_resume.pdf",
    missing_keywords=missing_keywords
)

✅ Optimised resume saved as: C:/Users/kambh/Downloads/optimised_resume.pdf


In [19]:
output_pdf_path = "C:/Users/kambh/Downloads/optimised_resume.pdf"

In [21]:
def generate_explanation(missing_keywords):
    explanation = f"""
✅ Changes Made:
- Added {len(missing_keywords)} missing keywords to improve ATS matching.
- Keywords added: {', '.join(missing_keywords)}
- A new page titled 'Optimised Skills' was appended to the resume.
- Resume is now better aligned with the job description.
"""
    return explanation

In [23]:
explanation = generate_explanation(missing_keywords)
print(explanation)


✅ Changes Made:
- Added 6 missing keywords to improve ATS matching.
- Keywords added: experience, looking, git, knowledge, plus, developer
- A new page titled 'Optimised Skills' was appended to the resume.
- Resume is now better aligned with the job description.



In [25]:
with open("C:/Users/kambh/Downloads/resume_changes.txt", "w", encoding="utf-8") as f:
    f.write(explanation)

In [27]:
def calculate_match_score(resume_text, job_keywords):
    resume_words = set(resume_text.lower().split())
    matched = [kw for kw in job_keywords if kw.lower() in resume_words]
    score = (len(matched) / len(job_keywords)) * 100
    return round(score, 2)

In [29]:
calculate_match_score(resume_text,job_keywords)

25.0

In [31]:
%%writefile resume_optimizer1.py
import streamlit as st
from resume_optimizer import extract_resume_text, extract_keywords, find_missing_keywords, append_keywords_to_pdf, generate_explanation
st.title("Resume Keyword Optimizer")

job_description = st.text_area("Paste Job Description Here")
resume_file = st.file_uploader("Upload Resume PDF", type=["pdf"])

if st.button("Optimize Resume"):
    if job_description and resume_file:
        resume_text = extract_resume_text(resume_file)
        job_keywords = extract_keywords(job_description)
        missing_keywords = find_missing_keywords(resume_text, job_keywords)

        explanation = generate_explanation(missing_keywords)
        st.write(explanation)

        output_path = "optimised_resume.pdf"
        append_keywords_to_pdf(resume_file, output_path, missing_keywords)

        with open(output_path, "rb") as f:
            st.download_button("Download Optimized Resume", f, file_name="optimised_resume.pdf")
    else:
        st.warning("Please provide both job description and resume.")

Overwriting resume_optimizer1.py


In [ ]:
!streamlit run resume_optimizer1.py